In [ ]:
# Package Loading
import numpy as np
from math import inf

from spotpython.utils.device import getDevice
from spotpython.utils.init import fun_control_init
from spotpython.hyperparameters.values import add_core_model_to_fun_control
from spotpython.hyperparameters.values import set_control_hyperparameter_value
from spotpython.utils.eda import gen_design_table
from spotpython.utils.init import design_control_init, surrogate_control_init
from spotpython.fun.hyperlight import HyperLight
from spotpython.spot import spot
from spotpython.utils.eda import gen_design_table
from spotpython.hyperparameters.values import get_tuned_architecture
from spotpython.light.loadmodel import load_light_from_checkpoint
from spotpython.light.cvmodel import cv_model
from torch.utils.data import DataLoader
from spotpython.utils.init import fun_control_init
from spotpython.hyperparameters.values import set_control_key_value
from spotpython.data.diabetes import Diabetes
from spotpython.light.regression.netlightregression import NetLightRegression
from spotpython.hyperdict.light_hyper_dict import LightHyperDict
from spotpython.hyperparameters.values import add_core_model_to_fun_control
from spotpython.hyperparameters.values import (
        get_default_hyperparameters_as_array, get_one_config_from_X)
from spotpython.plot.xai import (get_activations, get_gradients, get_weights,
                                 plot_nn_values_hist, plot_nn_values_scatter, visualize_weights,
                                 visualize_gradients, visualize_activations, visualize_gradient_distributions,
                                 visualize_weights_distributions)
from pyhcf.data.param_list_generator import (
    load_all_features_param_list,
    load_thermo_features_param_list,
    load_man_most_significant,
    load_man_significant,
)
from spotpython.light.predictmodel import predict_model
from spotpython.utils.file import save_experiment, load_experiment
from spotpython.data.lightdatamodule import LightDataModule
from spotpython.light.regression.netlightregression2 import NetLightRegression2
from spotpython.hyperdict.light_hyper_dict import LightHyperDict
from spotpython.hyperparameters.values import add_core_model_to_fun_control
from pyhcf.data.loadHcfData import loadFeaturesFromPkl
from pyhcf.data.param_list_generator import load_relevante_aero_variablen

In [ ]:
PREFIX="RELEVANTE_AERO_VARIABLEN_3551a_04a" # Prefix unbedingt ändern!
DATA_PKL_NAME = "RELEVANTE_AERO_VARIABLEN_DATA.pickle"
MAX_TIME = 300
FUN_EVALS = inf
FUN_REPEATS = 2
NOISE = False
OCBA_DELTA = 0
REPEATS = 2
INIT_SIZE = 20
WORKERS = 0
DEVICE = getDevice()
DEVICES = 1
TEST_SIZE = 0.3
K_FOLDS = 5
param_list= load_relevante_aero_variablen()
# param_list = load_all_features_param_list()
target = "N"
rmNA=True
rmMF=True
scale_data=True

In [ ]:

fun_control = fun_control_init(
    _L_in=len(param_list),
    _L_out=1,
    PREFIX=PREFIX,
    TENSORBOARD_CLEAN=True,
    device=DEVICE,
    enable_progress_bar=False,
    fun_evals=FUN_EVALS,
    fun_repeats=FUN_REPEATS,
    log_level=50,
    max_time=MAX_TIME,
    num_workers=WORKERS,
    ocba_delta = OCBA_DELTA,
    show_progress=True,
    test_size=TEST_SIZE,
    tolerance_x=np.sqrt(np.spacing(1)),
    verbosity=1,
    noise=NOISE
    )

In [ ]:
filename = DATA_PKL_NAME
dataset = loadFeaturesFromPkl(param_list=param_list, filename=filename, return_X_y=False)

In [ ]:
set_control_key_value(control_dict=fun_control,
                        key="data_set",
                        value=dataset,
                        replace=True)
print(len(dataset))

In [ ]:
add_core_model_to_fun_control(fun_control=fun_control,
                              core_model=NetLightRegression2,
                              hyper_dict=LightHyperDict)

In [ ]:
# Ändern der Default Hyperparameter 

set_control_hyperparameter_value(fun_control, "l1", [5, 9])
set_control_hyperparameter_value(fun_control, "epochs", [8, 12])
set_control_hyperparameter_value(fun_control, "batch_size", [8, 10])
set_control_hyperparameter_value(fun_control, "optimizer", [ 
    "Adagrad", "Adam"])
set_control_hyperparameter_value(fun_control, "dropout_prob", [0.001, 0.1])
set_control_hyperparameter_value(fun_control, "lr_mult", [0.01, 4])
set_control_hyperparameter_value(fun_control, "patience", [4, 7])
set_control_hyperparameter_value(fun_control, "act_fn",[
                "Sigmoid",
                "ReLU",
                "LeakyReLU",
             ] )
set_control_hyperparameter_value(fun_control, "initialization",["Default"] )
print(gen_design_table(fun_control))

In [ ]:
fun = HyperLight(log_level=50).fun

design_control = design_control_init(init_size=INIT_SIZE,
                                     repeats=REPEATS,)

surrogate_control = surrogate_control_init(noise=True,
                                            n_theta=2,
                                            min_Lambda=1e-6,
                                            max_Lambda=10,
                                            log_level=50,)

spot_tuner = spot.Spot(fun=fun,
                       fun_control=fun_control,
                       design_control=design_control,
                       surrogate_control=surrogate_control)
spot_tuner.run()

SPOT_PKL_NAME = save_experiment(spot_tuner, fun_control)

# tensorboard --logdir="runs/"

In [ ]:
if spot_tuner.noise:
    print(spot_tuner.min_mean_X)
    print(spot_tuner.min_mean_y)
else:
    print(spot_tuner.min_X)
    print(spot_tuner.min_y)

In [ ]:
spot_tuner.plot_progress(log_y=False)

In [ ]:
print(gen_design_table(fun_control=fun_control, spot=spot_tuner))

In [ ]:
# Get the Tuned Architecture 

config = get_tuned_architecture(spot_tuner, fun_control)
print(config)

# Predict on test data

In [ ]:
from spotpython.utils.file import load_experiment
from spotpython.light.predictmodel import predict_model
from spotpython.hyperparameters.values import get_one_config_from_X
from spotpython.plot.validation import plot_actual_vs_predicted
from sklearn.metrics import mean_squared_error

In [ ]:
# Prefix unbedingt anpassen, wenn ein altes Experiment geladen wird!
# Ansonsten wird das oben für das Tuning erstellte PREFIX verwendet.
# PREFIX="033allbartz09d" 
spot_tuner, fun_control = load_experiment(SPOT_PKL_NAME)

In [ ]:
print(f"noise: {spot_tuner.noise}")
if spot_tuner.noise:
    print(spot_tuner.min_mean_X)
    X = spot_tuner.to_all_dim(spot_tuner.min_mean_X.reshape(1,-1))
    print(spot_tuner.min_mean_y)
else:
    print(spot_tuner.min_X)
    X = spot_tuner.to_all_dim(spot_tuner.min_X.reshape(1,-1))
    print(spot_tuner.min_y)

print(f"X: {X}")
config = get_one_config_from_X(X, fun_control)
print(f"config: {config}")
batch_size = config["batch_size"]
print(f"batch_size: {batch_size}")

In [ ]:
# Training des Netzes mit den besten Hyperparametern
res = predict_model(config, fun_control)

In [ ]:
# Extraktion und Konvertieren der Ergebnisse in Numpy Arrays
x = res[0][0]
y = res[0][1]
yhat = res[0][2]
y_test = y.numpy().flatten()
print(y_test.shape)
y_pred = yhat.numpy().flatten()
print(y_pred.shape)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

In [ ]:
plot_actual_vs_predicted(y_test=y_test, y_pred=y_pred)

# Captum Analye

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from pyhcf.data.hcfDataModule import HCFDataModule
from pyhcf.data.loadHcfData import load_hcf_data
from pyhcf.utils.names import get_full_parameter_names
from spotpython.data.pkldataset import PKLDataset
from captum.attr import IntegratedGradients
from os import path
import matplotlib.pyplot as plt
from pyhcf.data.loadHcfData import loadFeaturesFromPkl
from pyhcf.data.param_list_generator import load_relevante_aero_variablen

In [ ]:
param_list= load_relevante_aero_variablen()
print(param_list)
len(param_list)

In [ ]:
filename = "aero_features.pickle"
dataset = loadFeaturesFromPkl(A=True, H=True, param_list=param_list, target="N", rmNA=True, rmMF=True,  scale_data=True,filename=filename, return_X_y=False)

In [ ]:
batch_size = 512
data_module = HCFDataModule(batch_size=batch_size, dataset = dataset, test_size=0.3)
train_set = data_module.train_dataloader()
test_set = data_module.test_dataloader()

In [ ]:
import torch.optim as optim
from torch import nn

from spotpython.utils.math import generate_div2_list

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        l1 = 256
        _L_in = len(param_list)
        _L_out = 1
        n_low = _L_in // 4
        # ensure that n_high is larger than n_low
        n_high = max(l1, 2 * n_low)
        hidden_sizes = generate_div2_list(n_high, n_low)
        dropout_prob = 0.03691049560954292

        # Create the network based on the specified hidden sizes
        layers = []
        layer_sizes = [_L_in] + hidden_sizes
        layer_size_last = layer_sizes[0]
        for layer_size in layer_sizes[1:]:
            layers += [
                nn.Linear(layer_size_last, layer_size),
                nn.BatchNorm1d(layer_size),
                nn.LeakyReLU(),
                nn.Dropout(dropout_prob),
            ]
            layer_size_last = layer_size
        print(f"layer_sizes w/o last, which is 1: {layer_sizes}")
        layers += [nn.Linear(layer_sizes[-1], _L_out)]
        # nn.Sequential summarizes a list of modules into a single module, applying them in sequence
        self.layers = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Performs a forward pass through the model.

        Args:
            x (torch.Tensor): A tensor containing a batch of input data.

        Returns:
            torch.Tensor: A tensor containing the output of the model.

        """
        x = self.layers(x)
        return x


model = CustomModel()
model

In [ ]:
# Define loss function and optimizer
criterion = nn.MSELoss()
lr_mult = 2.4863701285514677
# MANUALLY adjust lr according to the transformations shown in optimizer_handler from spotpython.hyperparameters.optimizer  
optimizer = optim.Adagrad(model.parameters(), lr=lr_mult * 0.01)

# For testing set epoch to 100
# epochs = 100
epochs = 4096

## Training des Netzes

In [ ]:
from spotpython.utils.device import getDevice
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = getDevice("cpu")
# device = getDevice()
print(device)
model.to(device)
train_losses = []
for epoch in range(epochs):
    print('epochs {}/{}'.format(epoch+1,epochs))
    model.train()
    running_loss = 0.0
    for idx, (inputs,labels) in enumerate(train_set):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        preds = model(inputs.float())
        labels = labels.view(len(labels), 1)
        loss = criterion(preds,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss

    train_loss = running_loss/len(train_set)
    # put the train loss tensor on the cpu and convert it to numpy:
    train_losses.append(train_loss.detach().cpu().numpy())

    print(f'train_loss {train_loss}')

In [ ]:
#| echo: false
#| label: fig-plot-train-aero-captum
#| fig-cap: Verlauf des Trainings-Losses
plt.plot(train_losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
model.eval()

In [ ]:
#| echo: false
#| label: aero-sig
n_rel = 31
def get_n_most_sig_features(model, test_set, batch_size, n_rel = n_rel, verbose=False):
    
    model.eval()
    total_attributions = None
    integrated_gradients = IntegratedGradients(model)


    for idx, (inputs, labels) in enumerate(test_set):
        #inputs = inputs.unsqueeze(0)

        # Ensure that the last batch is not smaller than the batch size
        # and only "full" batches are used for the analysis!
        if inputs.shape[0] != batch_size:
            continue    
        attributions, delta = integrated_gradients.attribute(inputs, return_convergence_delta=True)

        if total_attributions is None:
            total_attributions = attributions
        else:
            total_attributions += attributions

    # Calculation of average attribution across all batches
    avg_attributions = total_attributions.mean(dim=0).detach().numpy()
    
    # take the absolute value of the attributions
    abs_avg_attributions = np.abs(avg_attributions)

    # Get indices of the most important features
    top_n_indices = abs_avg_attributions.argsort()[-n_rel:][::-1]

    # Get the importance values for the top features
    top_n_importances = avg_attributions[top_n_indices]

    # Print the indices and importance values of the top features
    selected_indices = []
    if verbose:
        print(f"Die {n_rel} wichtigsten Features aus der Captum Analyse sind:")
    i = 1
    for idx, importance in zip(top_n_indices, top_n_importances):
        selected_indices.append(idx)
        if verbose:
            print(f"{i}. Feature Index: {idx}, Importance: {importance}")
        i += 1

    selected_significants = [param_list[i] for i in selected_indices]
    important = get_full_parameter_names(selected_significants)
    # print the elements of the list "important" in a single line
    if verbose:
        print("Die Namen der wichtigsten Features nach der Captum-Analyse sind: ", end="\n")
        for i in range(len(important)):
            if i < len(important) - 1:
                print(i+1, end=". ")
                print(important[i], end="\n")
            else:
                print(i+1, end=". ")
                print(important[i])

    # Final, total print:
    print(f"Die {n_rel} wichtigsten Features aus der Captum Analyse sind:")
    i = 1
    for idx, importance in zip(top_n_indices, top_n_importances):
        # selected_indices.append(idx)
        print(f"{i}. Feature: {important[i-1]}.  Index: {idx}, Importance: {importance}")
        i += 1
    

    return top_n_indices, top_n_importances, avg_attributions, selected_indices, important, selected_significants

In [ ]:
top_n_indices, top_n_importances, avg_attributions, selected_indices, important, selected_significants = get_n_most_sig_features(model, test_set, batch_size, n_rel = n_rel)

In [ ]:
param_list_aero = selected_significants
print(param_list_aero)

In [ ]:
def plot_sig_features(top_n_indices, top_n_importances, avg_attributions):
    # Visualize attributions using a bar plot
    # features = range(1, len(avg_attributions) + 1)
    # TODO: Check Indices!
    features = range(len(avg_attributions))
    plt.bar(features, avg_attributions)
    plt.title('Integrated Gradients - Relevanz der Variablen für die Vorhersage der Amplitude')
    plt.xlabel('Feature Index')
    plt.ylabel('Attribution Score')
    # add a grid each 5th feature
    plt.xticks(features[::5])
    plt.grid(alpha=0.25)
    plt.show()

In [ ]:
plot_sig_features(top_n_indices, top_n_importances, avg_attributions)